In [87]:
import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, Float, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker


from sqlalchemy.orm import configure_mappers
from sqlalchemy_schemadisplay import create_schema_graph
from sqlalchemy import MetaData

# Define the SQLAlchemy Database Model

## Proposed Schema
- **Property** (Core property information). Stores details specific to each property.
- **Location** (Details about the property location). Stores information such as country and other geographic details.
- **Price** (Price data for each property). Stores pricing information.
- **Feature** (Features such as average rooms, age, occupancy). Stores characteristics of the property such as median income in the area, number of rooms, etc.

In [89]:
# Define the database URL
DATABASE_URL = "sqlite:///real_estate.db"

# Create an engine
engine = create_engine(DATABASE_URL, echo=True)

# Define a base class for the declarative model
Base = declarative_base()



/var/folders/t_/k5bhrc1x419crrfg_t8zp5nm0000gp/T/ipykernel_47081/4272391801.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Property

In [91]:
# Property Table: Holds core property information
class Property(Base):
    __tablename__ = 'property'

    property_id = Column(Integer, primary_key=True, autoincrement=True)
    location_id = Column(Integer, ForeignKey('location.location_id'))
    feature_id = Column(Integer, ForeignKey('feature.feature_id'))
    price_id = Column(Integer, ForeignKey('price.price_id'))

    # Relationships
    location = relationship("Location", back_populates="properties")
    feature = relationship("Feature", back_populates="properties")
    price = relationship("Price", back_populates="properties")

### Location

In [93]:
# Location Table: Stores location details
class Location(Base):
    __tablename__ = 'location'

    location_id = Column(Integer, primary_key=True, autoincrement=True)
    country = Column(String, nullable=False)

    # Relationship to Property
    properties = relationship("Property", back_populates="location")

### Price

In [95]:
# Price Table: Stores price information
class Price(Base):
    __tablename__ = 'price'

    price_id = Column(Integer, primary_key=True, autoincrement=True)
    price = Column(Float, nullable=False)

    # Relationship to Property
    properties = relationship("Property", back_populates="price")

### Feature

In [97]:
class Feature(Base):
    __tablename__ = 'feature'

    feature_id = Column(Integer, primary_key=True, autoincrement=True)
    med_income = Column(Float, nullable=False)
    house_age = Column(Float, nullable=False)
    ave_rooms = Column(Float, nullable=False)
    ave_bedrooms = Column(Float, nullable=False)
    ave_occupancy = Column(Float, nullable=False)

    # Relationship to Property
    properties = relationship("Property", back_populates="feature")

## Finishing the DB setup

In [99]:
# Create the tables in the database
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

2024-10-17 14:36:51,020 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-17 14:36:51,022 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("property")
2024-10-17 14:36:51,024 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-17 14:36:51,025 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("property")
2024-10-17 14:36:51,026 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-17 14:36:51,027 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("location")
2024-10-17 14:36:51,028 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-17 14:36:51,029 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("location")
2024-10-17 14:36:51,030 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-17 14:36:51,031 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("price")
2024-10-17 14:36:51,032 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-17 14:36:51,033 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("price")
2024-10-17 14:36:51,033 INFO sqlalchemy.engine.Engine [raw sql] ()

## Create the ERD diagram

In [101]:
# Create engine
engine = create_engine("sqlite:///real_estate.db")

# Define metadata and reflect from engine
metadata = MetaData()
metadata.reflect(bind=engine)

# Configure mappers if using SQLAlchemy ORM
configure_mappers()

# Step 4: Create schema graph (add engine argument)
graph = create_schema_graph(
    metadata=metadata,
    engine=engine,
    show_datatypes=True,  # Show data types in the diagram
    show_indexes=True,  # Show index information in the diagram
    rankdir="LR",  # Layout from left to right
    concentrate=False  # No edge concentration
)

# Save the ER diagram to a file
graph.write_png("real_estate_schema.png")


## Insert Data into the New Tables

In [17]:
# Load the dataset
df = pd.read_csv('../data/clean/combined_dataset.csv')

In [19]:
# Dictionaries to store IDs for Location, Feature, and Price entries to avoid redundancy
location_map = {}
feature_map = {}
price_map = {}

In [ ]:
# Insert data into the new tables
properties = []
for index, row in df.iterrows():
    # Insert Location
    country = row['Country']
    if country not in location_map:
        location_entry = Location(country=country)
        session.add(location_entry)
        session.commit()
        location_map[country] = location_entry.location_id
    location_id = location_map[country]

    # Insert Feature
    feature_key = (row['MedInc'], row['HouseAge'], row['AveRooms'], row['AveBedrms'], row['AveOccup'])
    if feature_key not in feature_map:
        feature_entry = Feature(
            med_income=row['MedInc'],
            house_age=row['HouseAge'],
            ave_rooms=row['AveRooms'],
            ave_bedrooms=row['AveBedrms'],
            ave_occupancy=row['AveOccup']
        )
        session.add(feature_entry)
        session.commit()
        feature_map[feature_key] = feature_entry.feature_id
    feature_id = feature_map[feature_key]

    # Insert Price
    price = row['Price']
    if price not in price_map:
        price_entry = Price(price=price)
        session.add(price_entry)
        session.commit()
        price_map[price] = price_entry.price_id
    price_id = price_map[price]

    # Create Property entry
    property_entry = Property(
        location_id=location_id,
        feature_id=feature_id,
        price_id=price_id
    )
    properties.append(property_entry)
    
print("Data has been successfully inserted into the normalized database.")

2024-10-17 12:43:30,741 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-17 12:43:30,743 INFO sqlalchemy.engine.Engine INSERT INTO location (country) VALUES (?)
2024-10-17 12:43:30,744 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ('Slovakia',)
2024-10-17 12:43:30,746 INFO sqlalchemy.engine.Engine COMMIT
2024-10-17 12:43:30,749 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-17 12:43:30,752 INFO sqlalchemy.engine.Engine SELECT location.location_id AS location_location_id, location.country AS location_country 
FROM location 
WHERE location.location_id = ?
2024-10-17 12:43:30,753 INFO sqlalchemy.engine.Engine [generated in 0.00108s] (3,)
2024-10-17 12:43:30,756 INFO sqlalchemy.engine.Engine INSERT INTO feature (med_income, house_age, ave_rooms, ave_bedrooms, ave_occupancy) VALUES (?, ?, ?, ?, ?)
2024-10-17 12:43:30,756 INFO sqlalchemy.engine.Engine [generated in 0.00073s] (2.5634, 24.0, 3.0, 3.0, 3.6)
2024-10-17 12:43:30,758 INFO sqlalchemy.engine.Engine COMMIT
2024-